In [1]:
import spacy
nlp = spacy.load('es_core_news_lg')

In [4]:
import pandas as pd
data_path = '/lustre06/project/6001735/detests_2022/data/train.csv'
df = pd.read_csv(data_path).iloc[:15,:4]
df

,comment_id,sentence_pos,reply_to,sentence
0,0,1,0,La solución es desarrollar el pensamiento crít...
1,0,2,0,Hay que enseñar que la magia no existe.
2,0,3,0,Que todos los avances de la humanidad siempre ...
3,0,4,0,Enseñar en las escuelas la historia de las rel...
4,0,5,0,Desde las religiones de la edad de piedra hast...
5,0,6,0,Después de eso da igual lo que intenten adoctr...
6,0,7,0,"Pero no es posible prohibir las religiones, po..."
7,1,1,1,¿Manipulando?.
8,1,2,1,"Un hecho real, que no me impide NO demonizar a..."
9,1,3,1,No se donde ves la manipulación.


In [13]:
targets = ['xenophobia',
           'suffering',
           'economic',
           'migration',
           'culture',
           'benefits',
           'health',
           'security',
           'dehumanisation',
           'others',
          ]
for target in targets:
    print(f'[{target:14}] {df[df[target]==1].iloc[0,3].strip()} ' )
    print()

[xenophobia    ] la culpa es de occidente por eso no les queda mas remedio que inmolarse 

[suffering     ] Se metieron ellos y además a cambio de un pastizal y haciendo al puto asesino mafioso rico. 

[economic      ] Muy mal, el del zumo venía a pagar pensiones y por culpa de un asesino no va a poder cumplir con su sueño. 

[migration     ] Desgraciadamente, y visto lo visto, al final vamos a tener que aplicar lo que hace los israelíes en sus fronteras. 

[culture       ] Imagina un país como la India dónde hay millones de personas en castas inferiores que no protestan aunque los trates como ratas 

[benefits      ] Ni un duro de dinero público, ni permitir que se impongan sus absurdas normas en la sociedad y meter en la cárcel a todo aquel que realice un discurso de odio si son medidas posibles y efectivas. 

[health        ] En Cataluña trajeron a miles de musulmanes, subvencionados, voto cautivo y se les fue de las manos con los radicales, Menas, temporeros con virus, en Lérida, e

In [31]:
import pandas as pd
import numpy as np

rs = np.random.RandomState(0)
corr = pd.concat([df[targets],df.iloc[:,4:8]],axis=1).corr()
corr = np.abs(corr)
corr.style.background_gradient(cmap='coolwarm')

,xenophobia,suffering,economic,migration,culture,benefits,health,security,dehumanisation,others,racial_target,other_target,implicit,stereotype
xenophobia,1.000000,0.023422,0.060209,0.019660,0.041267,0.002450,0.004340,0.001119,0.054144,0.008672,0.091347,0.003644,0.099007,0.119321
suffering,0.023422,1.000000,0.053356,0.042246,0.001132,0.041861,0.053097,0.047693,0.030629,0.029660,0.186618,0.066188,0.185453,0.238249
economic,0.060209,0.053356,1.000000,0.050495,0.012937,0.058685,0.008087,0.011672,0.015915,0.000579,0.186804,0.006790,0.171433,0.222372
migration,0.019660,0.042246,0.050495,1.000000,0.035270,0.199175,0.050615,0.123080,0.084152,0.074511,0.457799,0.017017,0.398129,0.557281
culture,0.041267,0.001132,0.012937,0.035270,1.000000,0.047031,0.021006,0.209781,0.109969,0.024668,0.315560,0.051898,0.345905,0.419763
benefits,0.002450,0.041861,0.058685,0.199175,0.047031,1.000000,0.053674,0.066115,0.067141,0.144658,0.356298,0.013413,0.381345,0.439266
health,0.004340,0.053097,0.008087,0.050615,0.021006,0.053674,1.000000,0.029377,0.021607,0.021020,0.094710,0.003756,0.104669,0.123010
security,0.001119,0.047693,0.011672,0.123080,0.209781,0.066115,0.029377,1.000000,0.062102,0.068107,0.385766,0.022460,0.371537,0.492074
dehumanisation,0.054144,0.030629,0.015915,0.084152,0.109969,0.067141,0.021607,0.062102,1.000000,0.044086,0.194470,0.028780,0.202193,0.242065
others,0.008672,0.029660,0.000579,0.074511,0.024668,0.144658,0.021020,0.068107,0.044086,1.000000,0.193387,0.007506,0.260686,0.245827


In [40]:
from sklearn.preprocessing import normalize
import numpy as np
X = np.array([normalize([doc.vector])[0,:] for doc in nlp.pipe(df["sentence"])])
y = df[targets].values

print(X.shape)
print(y.shape)

(3817, 300)
(3817, 10)


In [29]:
for target in range(10):
    print(f'{targets[target]:18} {np.sum(y[:,target]):3} (de {len(y)}) ({np.sum(y[:,target])/len(y)*100:4.3f}%)')


xenophobia          16 (de 3817) (0.419%)
suffering           63 (de 3817) (1.651%)
economic            55 (de 3817) (1.441%)
migration          321 (de 3817) (8.410%)
culture            189 (de 3817) (4.952%)
benefits           206 (de 3817) (5.397%)
health              17 (de 3817) (0.445%)
security           255 (de 3817) (6.681%)
dehumanisation      65 (de 3817) (1.703%)
others              67 (de 3817) (1.755%)


,0
test_accuracy,0.938699
test_precision,0.108640
test_recall,0.340659
test_f1,0.158321


In [72]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import cross_validate
from sklearn.svm import SVC

results = np.zeros(shape=(10,8))
for target_idx in range(10):
    y_target = y[:,target_idx]
    assert X.shape==(3817, 300), X.shape
    assert y_target.shape==(len(X),)
    
    scores = cross_validate(SVC(kernel='rbf',class_weight='balanced'),X,y_target,cv=5,return_train_score=True, scoring=['accuracy','precision','recall','f1'],)
    values = pd.DataFrame(scores)[['train_accuracy','train_precision','train_recall','train_f1', 'test_accuracy','test_precision','test_recall','test_f1']].values
    values = np.average(values,axis=0)
    results[target_idx,:] = values


array([[0.99646327, 0.5552253 , 1.        , 0.71078821, 0.98952056,
        0.21666667, 0.11666667, 0.10666667],
       [0.95552778, 0.27169611, 1.        , 0.42703493, 0.93502535,
        0.09370454, 0.33333333, 0.14458758],
       [0.98080883, 0.43726703, 1.        , 0.6059703 , 0.96175472,
        0.21081081, 0.30909091, 0.21247888],
       [0.86180239, 0.37775056, 0.98830861, 0.54642487, 0.80378192,
        0.26743302, 0.73514423, 0.38879731],
       [0.8639641 , 0.26780842, 0.99735099, 0.42190721, 0.80275264,
        0.12733559, 0.47652916, 0.19957581],
       [0.90810889, 0.37126824, 1.        , 0.54112924, 0.86639059,
        0.25941228, 0.68420441, 0.36751857],
       [0.99823163, 0.7253429 , 1.        , 0.83823954, 0.99344932,
        0.36666667, 0.21666667, 0.27047619],
       [0.85466473, 0.31487051, 0.99411765, 0.47807632, 0.79251302,
        0.17837175, 0.57647059, 0.27222313],
       [0.94963415, 0.25541569, 1.        , 0.40613016, 0.92533983,
        0.05545774, 0.215384

In [74]:
pd.DataFrame(results,columns=['train_accuracy','train_precision','train_recall','train_f1', 'test_accuracy','test_precision','test_recall','test_f1'],index=targets)

,train_accuracy,train_precision,train_recall,train_f1,test_accuracy,test_precision,test_recall,test_f1
xenophobia,0.996463,0.555225,1.000000,0.710788,0.989521,0.216667,0.116667,0.106667
suffering,0.955528,0.271696,1.000000,0.427035,0.935025,0.093705,0.333333,0.144588
economic,0.980809,0.437267,1.000000,0.605970,0.961755,0.210811,0.309091,0.212479
migration,0.861802,0.377751,0.988309,0.546425,0.803782,0.267433,0.735144,0.388797
culture,0.863964,0.267808,0.997351,0.421907,0.802753,0.127336,0.476529,0.199576
benefits,0.908109,0.371268,1.000000,0.541129,0.866391,0.259412,0.684204,0.367519
health,0.998232,0.725343,1.000000,0.838240,0.993449,0.366667,0.216667,0.270476
security,0.854665,0.314871,0.994118,0.478076,0.792513,0.178372,0.576471,0.272223
dehumanisation,0.949634,0.255416,1.000000,0.406130,0.925340,0.055458,0.215385,0.087128
others,0.960440,0.309918,1.000000,0.472498,0.938699,0.108640,0.340659,0.158321


In [68]:
.shape

(8,)